In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
#nltk.download('punkt')
import pandas as pd
import numpy as np
#nltk.download('wordnet')
import itertools
import string
import os
from nltk.tokenize import RegexpTokenizer
import re

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


# Get the names and the scores

In [76]:
path = 'D:\\filings_clean_withscore'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))
scores = []
names = []
for f in files:
    scores.append(re.findall("_\d_\d_\d_\d", f))
    names.append(re.findall("text_10k_(.*)(?=_\d_\d_\d_)", f))

names_joined = [' '.join(x) for x in names]
text_dict  = {}

# Insert the text in a dictionary and create a dataframe

In [78]:
for file, name in zip(files, names_joined):
    with open(file, "r") as myfile:
        text_dict[name] = myfile.read().replace("\n", " ")
            

In [79]:
df_text = pd.DataFrame(text_dict.items(), columns=["id", "text"])

In [80]:
del text_dict, names

In [81]:
scores_joined = [' '.join(x) for x in scores]

In [86]:
del scores

# Create columns for the ESG scores

In [82]:
#scores_joined[0].split("_")[4]
df_text["total_esg"] = 0
df_text["E_score"] = 0
df_text["S_score"] = 0
df_text["G_score"] = 0

In [84]:
for row in range(len(df_text)):
    df_text["total_esg"].iloc[row] = scores_joined[row].split("_")[1]
    df_text["E_score"].iloc[row] = scores_joined[row].split("_")[2]
    df_text["S_score"].iloc[row] = scores_joined[row].split("_")[3]
    df_text["G_score"].iloc[row] = scores_joined[row].split("_")[4]
    

In [95]:
df_text.head(2)

,text,total_esg,E_score,S_score,G_score
id,,,,,
1000180_2010,10-K 1 form_10k.htm FORM 10-K FY09 form_10k.ht...,1,1,2,1
1000180_2011,10-K 1 form_10k.htm FORM 10-K FY10 form_10k.ht...,1,1,2,2


In [94]:
df_text.set_index("id", inplace=True)

In [99]:
df_text

,text,total_esg,E_score,S_score,G_score
id,,,,,
1000180_2010,10-K 1 form_10k.htm FORM 10-K FY09 form_10k.ht...,1,1,2,1
1000180_2011,10-K 1 form_10k.htm FORM 10-K FY10 form_10k.ht...,1,1,2,2
1000180_2012,10-K 1 sndk201110-k.htm FORM 10-K FY11 SNDK 20...,1,1,2,1
1000180_2013,10-K 1 sndk201210-k.htm FORM 10-K FY12 SNDK 20...,1,1,2,1
1000180_2014,10-K 1 sndk201310-k.htm FORM 10-K FY13 SNDK 20...,2,2,1,2
1000180_2015,10-K 1 sndk201410-k.htm FORM 10-K FY14 SNDK 20...,2,2,1,2
1000228_2010,10-K 1 the10k_2009.htm THE 2009 ANNUAL 10-K RE...,2,2,1,2
1000228_2011,10-K 1 the2010_10k.htm THE 2010 ANNUAL 10-K RE...,1,1,2,1
1000228_2012,10-K 1 the10k_2011.htm THE 2011 ANNUAL 10-K RE...,1,1,2,1


# Text preprocessing

In [100]:
from nltk.stem import WordNetLemmatizer

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(df_text)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(df_text["text"][sen]))

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)

    # Converting to Lowercase
    document = document.lower()

    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    documents.append(document)

# Insert the preprocessed documents in the dataframe

In [103]:
for row_t in range(len(df_text)):
    df_text["text"].iloc[row_t] = documents[row_t]

# Split into train and test set

In [106]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_text['text'], df_text['total_esg'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# Bag of words

In [107]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df_text['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

# Term Frequency - Inverse Document Frequency (TF-IDF)
* Word
* ngrams
* Characters

In [108]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df_text['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df_text['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df_text['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

# Word embeddings

In [111]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('crawl-300d-2M.vec', encoding='utf-8')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df_text['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Feature engineering
## Groups of words need to be added

In [113]:
df_text['char_count'] = df_text['text'].apply(len)
df_text['word_count'] = df_text['text'].apply(lambda x: len(x.split()))
df_text['word_density'] = df_text['char_count'] / (df_text['word_count']+1)
df_text['punctuation_count'] = df_text['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 



In [140]:
#d = pd.DataFrame(xtrain_tfidf_ngram_chars.toarray(), columns=tfidf_vect_ngram_chars.get_feature_names())
#train_x

In [146]:
#nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hella\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

# Latent Dirichlet Allocation (LDA)

In [ ]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

# Recurrent Neural Network LSTM

In [2]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [3]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print "RNN-LSTM, Word Embeddings",  accuracy